In [53]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize,word_tokenize
import numpy as np
import re
import numpy
from numpy import delete
import math
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
def read_article(para):
    file = open( r"C:\Users\user\009.txt",encoding='utf-8')
    filedata = file.read()
#     print(filedata)
    sentences= sent_tokenize(filedata)
    #print("Tokenized Sentences: ",sentences)
    sent=word_tokenize(filedata)
#     print(sentences)
    return sentences
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    #ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        #print(words)
        for word in words:
            word = word.lower()
            #word = ps.stem(word)
            #print(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
         
        tf_matrix[sent] = tf_table

    return tf_matrix


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))
            
        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    sum=0
    text=[]
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)
            
        
#             print(tf_idf_table[word1])
        tf_idf_matrix[sent1]= tf_idf_table
#         print(tf_idf_matrix[sent1])
        
    return tf_idf_matrix
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    text = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        text[sent] = total_score_per_sentence / count_words_in_sentence
#     print("tf_idf score for all the sentences",text)   
    return text
def cosine_value(sentences):
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(sentences)


# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=[sentences])
    df
    cosine=cosine_similarity(df, df)
#     print(cosine)
    return cosine

def average_value(final_score): 

    return sum(final_score) / len(final_score)
    

def cal_pop_fitness(text, pop):
    n=[]
    num=[]
    arr=[]
    arr1=[]
    final=[]
    final1=[]
    final2=[]
    final_score=[]
    file = open( r"C:\Users\user\009.txt",encoding='utf-8')
    filedata = file.read()
    sentences = sent_tokenize(filedata)
    cosine=cosine_value(sentences)
    x=np.array(cosine)
    y=np.argwhere(x>0.5)
#     print(y)
    k=0
    for i in range(len(y)): 
        if(y[i][k] ==y[i][k+1]): 
            continue

        elif(text[y[i][k]]>text[y[i][k+1]]):
            n.append(text[y[i][k]])
            arr.append(y[i][k])
            arr.append(y[i][k+1])
            num.append(y[i][k])
          
        else:
            n.append(text[y[i][k+1]])
            num.append(y[i][k+1])
#     print(np.unique(n))#returning the greater score
    arr1=np.unique(arr)
#     print("arr1------",arr1)
#     print("score====",n)#returning the comparing sentences score
    for i in range(len(y)):
        if (y[i][k] not in arr1):
            final.append(y[i][k])
#     print("other===",final)#returning the other sentences
    final1=final+num
    final2=np.unique(final1)
#     print("Remaining sentences index =============", final2)
    for i in range(len(final2)):
        final_score.append(text[final2[i]])
#     print("remaining sentence score",score)# returning the other sentence score
#     final_score=list(set(final_list))
    average=average_value(final_score)
    print("Average value--",average)
#     print("Call pop fitness return==",final_score)
    j2 = [i for i in final_score if i >= average]
    j3=list(j2)
#     print("score greater than avg value", j3)
#    list(dict.fromkeys(mylist))
    return j3

  
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover
def generate_summary(para, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    

    # Step 1 - Read text anc split it
    sentences =  read_article(para)
    
    total_documents = len(sentences)
    
    freq_matrix = _create_frequency_matrix(sentences)
    #print("freqency_matrix",freq_matrix)

    
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print("tf_matrix:",tf_matrix)
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print("idf_matrix",idf_matrix)

    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print("\n\n")
    #print("tf-idf: ",tf_idf_matrix)
    sentence_scores = _score_sentences(tf_idf_matrix)
#     print(sentence_scores)
    text= list(sentence_scores.values())
    print("Input_values--",text)
    print("\n")
    num_weights=6
    sol_per_pop = len(sentence_scores)
    num_parents_mating = 6
    # Defining the population size.
    pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
#Creating the initial population.
    new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
    print("New population--",new_population)
    print("\n")
    num_generations = 5
    for generation in range(num_generations):
        print("Generation : ", generation)
    # Measing the fitness of each chromosome in the population.
        fitness = cal_pop_fitness(text, new_population)
       
        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(new_population, fitness, num_parents_mating)
        print("parents---",parents)
        print("\n")

    # Generating next generation using crossover.
        offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

        print("offspring_crossover----", offspring_crossover)
        print("\n")
    # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover)
        print("offspring_mutation---", offspring_mutation)
    # Creating the new population based on the parents and offspring.
        new_population[0:parents.shape[0], :] = parents
        new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
#         print("Best result : ", numpy.max(numpy.sum(new_population*text)))
#     print("Best result : ", numpy.max(fitness))#numpy.sum(new_population*text, axis=1)))

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
#     text=_tf_idf(tf_matrix, idf_matrix)
    fitness = cal_pop_fitness(text, new_population)
    
    #extracting the keys from values 
    p= ([list(sentence_scores.keys())[list(sentence_scores.values()).index(fitness[i])]for i  in range(len(fitness))])
    print(p)
    f=open("output1.txt",'a')
    f.write(str(p))

   

# Then return the index of that solution corresponding to the best fitness.
#     best_match_idx = numpy.where(fitness == numpy.max(fitness))
    print("\n")
    print ("Best solution : ", fitness)#values grater than average value
    print("\n")
    
    

    
    print("Best solution fitness : ", numpy.max(fitness))



generate_summary( "msft.txt", 2)

Input_values-- [0.08624087872598776, 0.08945701283675807, 0.07369525307111928, 0.18096024860839166, 0.09113520205263173, 0.11638067249993161, 0.12987183640810981, 0.08888531509563562, 0.08790025332783148, 0.05104589654293336, 0.09912934995875422, 0.13048678535406866, 0.09230496633987025, 0.35879860897450916, 0.1193097235921158, 0.07874560848934163, 0.07546900556522924, 0.29406191834764567, 0.19519948058501743, 0.3932413687874989, 0.23193340701746265, 0.20928546093957512, 0.3899561812889657, 0.18683833680849635, 0.1602041839494338, 0.3175585255330126, 0.362008804330077, 0.14860149256877414, 0.36623714115568834, 0.4935847006754926, 0.692541112035929, 0.3932413687874989, 0.15257635359744373, 0.29737102015358025, 0.15989195520988678, 0.16722470391625716, 0.0986913950699312, 0.3183253460301813, 0.3466714457050571, 0.09746642133987031, 0.13036834228618685, 0.26087803028488815, 0.22476562748810772, 0.14106217559054965, 0.07825428943254939, 0.2593247590322883, 0.16489814989649065, 0.1537563548